<a href="https://colab.research.google.com/github/DmitriyValetov/nlp_course_project/blob/master/ria_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [0]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Download a file based on its file ID.
# 1-UtATnzLE809Vi6RLgy3GRHX2TXRzhd6  # norm
# 1bhsdkXYEe4qixPddK9DkaQ-7z0jAn5Bi  # stop
# 1-40QXRckYZIfJTPiAhtQ8LAeBKks8ITx  # lem
# 1m7unmZmh0B3DJ-hiLm8-a5WoXECMc4cV  # snow
file_id = '1-40QXRckYZIfJTPiAhtQ8LAeBKks8ITx'
downloaded = drive.CreateFile({'id': file_id})
downloaded.FetchMetadata()
fn = downloaded.metadata['title']
print(f'downloading: {fn}')
downloaded.GetContentFile(fn)

In [0]:
from google.colab import drive
import os

drive.mount('/content/drive')
root = '/content/drive/My Drive/'
data_fn = 'stop_lem_norm_sents_ria.json.gz'
data_path = os.path.join(root, data_fn)
print(f'Check {data_path}: {os.path.exists(data_path)}')

In [0]:
import gzip
import json
from tqdm.notebook import tqdm

# data_path = 'stop_lem_norm_sents_ria.json.gz'

# first check
with gzip.open(data_path, 'rb') as f:
  print(json.loads(next(f)))

# all check
# with gzip.open(fn, 'rb') as f:
#   for line in tqdm(f):
#     json.loads(line)

# all check bad
cnt = 0
bad_cnt = 0
with gzip.open(data_path, 'rb') as f:
  for line in tqdm(f):
    cnt += 1
    n = json.loads(line)
    if len(n['text']) == 0 or len(n['title']) == 0:
      bad_cnt += 1
      print(cnt)
      print(n)
print(f'{bad_cnt}/{cnt}')

In [71]:
from collections import Counter
import os
import gzip
import json
from tqdm.notebook import tqdm

vocab = {}
cnt, bad_ids = 0, []
_, ext = os.path.splitext(data_path)
max_samples = 1003869  # max_samples=1003869
with gzip.open(data_path, 'rb') if ext == '.gz' else open(data_path) as f:
  for _ in tqdm(range(max_samples), desc='loading samples'):
    n = json.loads(next(f))
    if not len(n['text']) == 0 and not len(n['title']) == 0:
      for s in n['text']:
        for w in s.split():
          vocab[w] = vocab.setdefault(w, 0) + 1
      for s in n['title']:
        for w in s.split():
          vocab[w] = vocab.setdefault(w, 0) + 1
    else:
      bad_ids.append(cnt)
    cnt += 1
print(f'bad texts: {len(bad_ids)} {bad_ids}')
print(f'unique: {len(vocab)}, total: {sum(vocab.values())}')


bad texts: 146 [85124, 121073, 167715, 176024, 191679, 196491, 233761, 240737, 248487, 260013, 285129, 292992, 293089, 293116, 301604, 305587, 312586, 314213, 316134, 317722, 318000, 318655, 325713, 362998, 366397, 382262, 397968, 400780, 406416, 411736, 431649, 437910, 453488, 463202, 463257, 467341, 471837, 477691, 480756, 483624, 487196, 493816, 495050, 495318, 495338, 495705, 495898, 498775, 499475, 501329, 503532, 503880, 504118, 504486, 504600, 508333, 510257, 517735, 518394, 519404, 519432, 519506, 520672, 521438, 521730, 524884, 525492, 526160, 526173, 527055, 527102, 527562, 529467, 529469, 530575, 531190, 532894, 533960, 533982, 537247, 540254, 540565, 542038, 542044, 542769, 544866, 545213, 550244, 552215, 557118, 570332, 571466, 572783, 572920, 573314, 574084, 575678, 575691, 583560, 612454, 666619, 680547, 681071, 690771, 693126, 778882, 781772, 794032, 795809, 804124, 819613, 822100, 858471, 868287, 906565, 908647, 909081, 909141, 911907, 912070, 914943, 915590, 916612, 

In [72]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import json
import gzip
from pprint import pprint
from collections import Counter
from tqdm.notebook import tqdm
import numpy as np
import gc

class RiaDataset(Dataset):
  def __init__(self, path, max_vocab=10000, max_samples=500, vocab=None,
               special_chars=('<pad>', '<unk>', '<go>', '<eos>')):
    super(RiaDataset).__init__()
    if vocab is None:
      vocab = Counter() # word: counter
      cnt, bad_ids, samples = 0, [], []
      _, ext = os.path.splitext(path)
      with gzip.open(path, 'rb') if ext == '.gz' else open(path) as f:
        for _ in tqdm(range(max_samples), desc='loading samples'):
          n = json.loads(next(f))
          if not len(n['text']) == 0 and not len(n['title']) == 0:        
            samples.append(n)
            vocab.update(w for s in n['text'] for w in s.split())
            vocab.update(w for s in n['title'] for w in s.split())
          else:
            bad_ids.append(cnt)
          cnt += 1
      print(f'bad texts: {len(bad_ids)} {bad_ids}')
      print('Vocabulary')
      print(f'unique: {len(vocab)}, total: {sum(vocab.values())}')
      self.itos = list(special_chars)
      top_words = vocab.most_common(max_vocab)
      self.itos += [w for w, _ in top_words]
      self.stoi = {x: i for i, x in enumerate(self.itos)}
      assert len(self.itos) == len(self.stoi)
      print('Reduced vocabulary')
      print(f'unique: {len(self.stoi)}, total: {sum(c for _, c in top_words)}')
      print('Encoding samples')
      self.samples = [] # [list of list np arrays, list of list of np arrays]
      for s in tqdm(samples, desc='encoding samples'):
        x1 = [self.encode(x) for x in s['text']]
        x2 = [self.encode(x) for x in s['title']]
        self.samples.append([x1, x2])
      print(f'samples: {len(self.samples)}')
    else:
      if not isinstance(vocab, Counter):  # if dict cast to Counter
        vocab = Counter(vocab)
      print('Vocabulary')
      print(f'unique: {len(vocab)}, total: {sum(vocab.values())}')
      self.itos = list(special_chars)
      top_words = vocab.most_common(max_vocab)
      self.itos += [w for w, _ in top_words]
      self.stoi = {x: i for i, x in enumerate(self.itos)}
      assert len(self.itos) == len(self.stoi)
      print('Reduced vocabulary')
      print(f'unique: {len(self.stoi)}, total: {sum(c for _, c in top_words)}')
      print('Encoding samples')
      self.samples = [] # [list of list np arrays, list of list of np arrays]
      cnt, bad_ids = 0, []
      _, ext = os.path.splitext(path)
      with gzip.open(path, 'rb') if ext == '.gz' else open(path) as f:
        for _ in tqdm(range(max_samples), desc='loading samples'):
          n = json.loads(next(f))
          if not len(n['text']) == 0 and not len(n['title']) == 0:        
            x1 = [self.encode(x) for x in n['text']]
            x2 = [self.encode(x) for x in n['title']]
            self.samples.append([x1, x2])
          else:
            bad_ids.append(cnt)
          cnt += 1
      print(f'bad texts: {len(bad_ids)} {bad_ids}')
      print(f'samples: {len(self.samples)}')

  def __len__(self):
    return len(self.samples)

  def __getitem__(self, i):
    return self.samples[i]

  def encode(self, s):
    return np.array([2] + [self.stoi.get(x, 1) for x in s.split()] + [3])

  def decode(self, s):
    return [self.itos[x] for x in s]

class Collate():
  def __init__(self, n_x1=1):
     self.n_x1 = n_x1

  def __call__(self, batch):
    bx1, bx2 = [], []
    for b in batch:
      x1s, x2s = b
      # skip first sentence (usually date and place)
      ei = min(len(x1s), 1 + self.n_x1)
      si = 1 if ei > 1 else 0
      # print(self.n_x1, si, ei, len(x1s))
      for x1 in x1s[si:ei]:
        for x2 in x2s:
          bx1.append(torch.as_tensor(x1))
          bx2.append(torch.as_tensor(x2))
    bx1 = torch.nn.utils.rnn.pad_sequence(bx1, batch_first=True)
    bx2 = torch.nn.utils.rnn.pad_sequence(bx2, batch_first=True)
    batch = [bx1, bx2]
    return batch

ds = RiaDataset(data_path, max_vocab=50000, max_samples=1003869, vocab=vocab) 
# max_samples=1003869 max_vocab=50000
# unique words: 601956, total words: 198199252
# vocabulary: 50004, words in vocabulary: 194385367
torch.manual_seed(0)
torch.cuda.manual_seed(0)
# dl = DataLoader(ds, batch_size=1, num_workers=1, shuffle=False, 
#                 drop_last=False, collate_fn=Collate())
# for s in tqdm(dl, desc='dataset'):
#   bx1, bx2 = s
#   assert bx1.size()[0] == bx2.size()[0]
#   # print(bx1.size(), bx2.size())
#   # for x1, x2 in zip(bx1, bx2):
#   #   print(ds.decode(x1), ds.decode(x2))
#   # print([ds.decode(x) for x in bx1])
#   # print([ds.decode(x) for x in bx2])
train_len = int(0.7*len(ds))
test_len = int(0.2*len(ds))
val_len = len(ds) - train_len - test_len
lens = [train_len, test_len, val_len]
print(lens, sum(lens), len(ds))
train_ds, test_ds, val_ds = random_split(ds, lens)
train_dl = DataLoader(train_ds, batch_size=2, num_workers=1, 
                      shuffle=True, drop_last=False, 
                      collate_fn=Collate(n_x1=1))
test_dl = DataLoader(test_ds, batch_size=3, num_workers=1, 
                     shuffle=False, drop_last=False, 
                     collate_fn=Collate(n_x1=1))
val_dl = DataLoader(val_ds, batch_size=3, num_workers=1, 
                    shuffle=False, drop_last=False, 
                    collate_fn=Collate(n_x1=1))
# for s in tqdm(train_dl, desc='train'):
#   bx1, bx2 = s
#   assert bx1.size()[0] == bx2.size()[0]
#   # print(bx1.size(), bx2.size())
# for s in tqdm(test_dl, desc='test'):
#   bx1, bx2 = s
#   assert bx1.size()[0] == bx2.size()[0]
#   # print(bx1.size(), bx2.size())
# for s in tqdm(val_dl, desc='val'):
#   bx1, bx2 = s
#   assert bx1.size()[0] == bx2.size()[0]
#   # print(bx1.size(), bx2.size())
gc.collect()

Vocabulary
unique: 601956, total: 198199252
Reduced vocabulary
unique: 50004, total: 194385367
Encoding samples



bad texts: 146 [85124, 121073, 167715, 176024, 191679, 196491, 233761, 240737, 248487, 260013, 285129, 292992, 293089, 293116, 301604, 305587, 312586, 314213, 316134, 317722, 318000, 318655, 325713, 362998, 366397, 382262, 397968, 400780, 406416, 411736, 431649, 437910, 453488, 463202, 463257, 467341, 471837, 477691, 480756, 483624, 487196, 493816, 495050, 495318, 495338, 495705, 495898, 498775, 499475, 501329, 503532, 503880, 504118, 504486, 504600, 508333, 510257, 517735, 518394, 519404, 519432, 519506, 520672, 521438, 521730, 524884, 525492, 526160, 526173, 527055, 527102, 527562, 529467, 529469, 530575, 531190, 532894, 533960, 533982, 537247, 540254, 540565, 542038, 542044, 542769, 544866, 545213, 550244, 552215, 557118, 570332, 571466, 572783, 572920, 573314, 574084, 575678, 575691, 583560, 612454, 666619, 680547, 681071, 690771, 693126, 778882, 781772, 794032, 795809, 804124, 819613, 822100, 858471, 868287, 906565, 908647, 909081, 909141, 911907, 912070, 914943, 915590, 916612, 

0

# Model

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class EncoderRNN(nn.Module):
  def __init__(self, num_embeddings, embedding_dim, padding_idx,
               rnn_type, hidden_size, num_layers=1, rnn_dropout=0,
               bidirectional=False, dropout=0, pack=False):
    super(EncoderRNN, self).__init__()
    rnn_map = {'RNN': nn.RNN, 'GRU': nn.GRU, 'LSTM': nn.LSTM}
    self.embedding = nn.Embedding(num_embeddings=num_embeddings, 
                                  embedding_dim=embedding_dim,
                                  padding_idx=padding_idx)
    self.rnn = rnn_map[rnn_type](input_size=embedding_dim, 
                                 hidden_size=hidden_size,
                                 num_layers=num_layers,
                                 batch_first=True,
                                 dropout=rnn_dropout,
                                 bidirectional=bidirectional)
    self.dropout = nn.Dropout(dropout)
    self.pack = pack

  def forward(self, x1):  # [B, L]
    x = self.embedding(x1)  # [B, L] -> [B, L, E]
    if self.pack:  # [B, L, E] -> Packed [B, L, E]
      ls = torch.sum(x1 != 0, dim=1)
      x = pack_padded_sequence(x, ls, batch_first=True, enforce_sorted=False)
    # [B, L, E] -> (Packed) [B, L, ND*H], ([NL*ND, B, H], ([NL*ND, B, H]))
    if isinstance(self.rnn, nn.LSTM):
      ht, (hn, cn) = self.rnn(x)
      hn, cn = self.dropout(hn), self.dropout(cn)
      return ht, (hn, cn)
    else:
      ht, hn = self.rnn(x)
      hn = self.dropout(hn)
      return ht, hn


class AttentionDecoderRNN(nn.Module):
  def __init__(self, num_embeddings, embedding_dim, padding_idx,
               rnn_type, hidden_size, num_layers=1, rnn_dropout=0,
               bidirectional=False, dropout=0, out_hidden=0,
               attn_type='soft_dot', pack=False):
    super(AttentionDecoderRNN, self).__init__()
    rnn_map = {'RNN': nn.RNN, 'GRU': nn.GRU, 'LSTM': nn.LSTM}
    attn_types = ['dot', 'cos', 'dist', 'soft_dot', 'soft_cos', 'soft_dist', 'none']
    self.attn_type = attn_type
    self.embedding = nn.Embedding(num_embeddings=num_embeddings, 
                                  embedding_dim=embedding_dim,
                                  padding_idx=padding_idx)
    self.rnn = rnn_map[rnn_type](input_size=embedding_dim, 
                                 hidden_size=hidden_size,
                                 num_layers=num_layers,
                                 batch_first=True,
                                 dropout=rnn_dropout,
                                 bidirectional=bidirectional)
    out_input = hidden_size if self.attn_type == 'none' else 2*hidden_size
    if out_hidden > 0:
      self.out_hidden = nn.Linear(out_input, out_hidden)
      self.out = nn.Linear(out_hidden, num_embeddings)
    else:
      self.out_hidden = None
      self.out = nn.Linear(out_input, num_embeddings)
    self.dropout = nn.Dropout(dropout)
    self.softmax = nn.LogSoftmax(dim=2)
    self.pack = pack

  def forward(self, x2, h1, x1):  # [B, L], [B, L, ND*H], ([NL*ND, B, H], [NL*ND, B, H])
    x = self.embedding(x2)  # [B, L] -> [B, L, E]
    if self.pack:  # [B, L, E] -> Packed [B, L, E]
      lengths = torch.sum(x2 != 0, dim=1)
      x = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
    ht1, hns1 = h1  # [B, L, ND*H], ([NL*ND, B, H], [NL*ND, B, H])
    # [B, L, D], ([NL*ND, B, H], [NL*ND, B, H]) -> [B, L, ND*H], ([NL*ND, B, H], [NL*ND, B, H])
    ht2, hns2 = self.rnn(x, hns1)
    if self.pack:  # Packed [B, L, E] -> [B, L, E]
      ht1, _ = pad_packed_sequence(ht1, batch_first=True)
      ht2, _ = pad_packed_sequence(ht2, batch_first=True)
    if self.attn_type == 'none':
      if self.rnn.bidirectional:
        B2, L2, NDH2 = ht2.size()
        # [B, L, ND*H] -> [B, L, ND, H] -> [B, L, H]
        ht2 = ht2.view(B2, L2, 2, int(NDH2/2)).mean(2)
      x = self.dropout(ht2)
      if self.out_hidden is not None:
        x = self.out_hidden(x)
      x = self.out(x)
      y2 = self.softmax(x)
      return y2, None
    else:
      if self.rnn.bidirectional:
        B1, L1, NDH1 = ht1.size()
        B2, L2, NDH2 = ht2.size()  # B1 == B2, NDH1 == NDH2
        # [B, L, ND*H] -> [B, L, ND, H] -> [B, L, H]
        ht1 = ht1.view(B1, L1, 2, int(NDH1/2)).mean(2)
        ht2 = ht2.view(B2, L2, 2, int(NDH2/2)).mean(2)
      # [L2, H], [L1, H] -> [L2, L1]
      # mask where x1 and x2 token is <PAD>
      # pad_mask = torch.einsum('bi,bj->bij', x2, x1) == 0
      # mask where only x1 token is <PAD>
      pad_mask = torch.einsum('bi,bj->bij', torch.ones_like(x2), x1) == 0
      # [B, L2, H], [B, L1, H] -> [B, L2, L1]  # attention
      # [B, L2, L1], [B, L1, H] -> [B, L2, H]  # weighted h1
      if self.attn_type == 'dot':
        attn = torch.einsum('bih,bjh->bij', ht2, ht1)  # dot product
        attn[pad_mask] = 0
      elif self.attn_type == 'soft_dot':
        attn = torch.einsum('bih,bjh->bij', ht2, ht1)  # dot product
        attn[pad_mask] = float('-inf')
        attn = F.softmax(attn, 2)
      elif self.attn_type == 'dist':
        ht1, ht2 = ht1.contiguous(), ht2.contiguous()
        attn = torch.cdist(ht2, ht1)  # euclidian distance
        attn = torch.masked_fill(attn, pad_mask, float('inf'))
        attn = F.threshold(attn, threshold=1e-6, value=1e-6)  # short dist
        attn = 1/attn  # inverse
        attn = F.normalize(attn, p=1, dim=2)  # to [0, 1]
      elif self.attn_type == 'soft_dist':
        ht1, ht2 = ht1.contiguous(), ht2.contiguous()
        attn = torch.cdist(ht2, ht1)  # euclidian distance
        attn = torch.masked_fill(attn, pad_mask, float('inf'))
        attn = F.softmin(attn, 2)
      elif self.attn_type == 'cos':
        ht1n = F.normalize(ht1, p=2, dim=2)  # normalize to length = 1
        ht2n = F.normalize(ht2, p=2, dim=2)  # normalize to length = 1
        attn = torch.einsum('bih,bjh->bij', ht2n, ht1n)  # dot product
        attn[pad_mask] = 0
      elif self.attn_type == 'soft_cos':
        ht1n = F.normalize(ht1, p=2, dim=2)  # normalize to length = 1
        ht2n = F.normalize(ht2, p=2, dim=2)  # normalize to length = 1
        attn = torch.einsum('bih,bjh->bij', ht2n, ht1n)  # dot product
        attn[pad_mask] = float('-inf')
        attn = F.softmax(attn, 2)
      hw1 = torch.einsum('bij,bjh->bih', attn, ht1)  # weighted h1
      ha = torch.cat((ht2, hw1), 2)  # [B, L2, H], [B, L2, H] -> [B, L2, H+H]
      x = self.dropout(ha)  # [B, L2, H+H] -> [B, L2, H+H]
      if self.out_hidden is not None:
        x = self.out_hidden(x)   # [B, L2, H+H] -> [B, L2, OH]
      x = self.out(x)  # [B, L2, H+H] or [B, L2, OH] -> [B, L2, D2]
      y2 = self.softmax(x)  # [B, L2, D2] -> [B, L2, D2]
      return y2, attn


class EncoderDecoder(nn.Module):
  def __init__(self, 
               enc_num_embeddings, enc_embedding_dim, enc_padding_idx,
               dec_num_embeddings, dec_embedding_dim, dec_padding_idx,
               rnn_type, hidden_size, num_layers=1, out_hidden=0,
               enc_rnn_dropout=0, dec_rnn_dropout=0,
               bidirectional=False, enc_dropout=0, dec_dropout=0, 
               attn_type='dot', pack=False):
    super(EncoderDecoder, self).__init__()
    self.encoder = EncoderRNN(num_embeddings=enc_num_embeddings, 
                              embedding_dim=enc_embedding_dim, 
                              padding_idx=enc_padding_idx,
                              hidden_size=hidden_size, 
                              rnn_type=rnn_type, 
                              bidirectional=bidirectional,
                              num_layers=num_layers,
                              dropout=enc_dropout,
                              rnn_dropout=enc_rnn_dropout,
                              pack=pack)
    self.decoder = AttentionDecoderRNN(num_embeddings=dec_num_embeddings, 
                                       embedding_dim=dec_embedding_dim, 
                                       padding_idx=dec_padding_idx,
                                       hidden_size=hidden_size, 
                                       rnn_type=rnn_type, 
                                       bidirectional=bidirectional,
                                       num_layers=num_layers,
                                       dropout=dec_dropout,
                                       out_hidden=out_hidden,
                                       rnn_dropout=dec_rnn_dropout,
                                       attn_type=attn_type,
                                       pack=pack)

  def forward(self, x1, x2):
    #  [B, L1] -> [B, L1, ND*H], ([NL*ND, B, H], [NL*ND, B, H])
    h1 = self.encoder(x1)
    # [B, L2], ([B, L1, ND*H], ([NL*ND, B, H], [NL*ND, B, H])) -> [B, L2, E2]
    y2 = self.decoder(x2, h1, x1)
    return y2


def external_attn(x1, x2, attn_dict):
    attn_dict = {1: [1], 2: [2], 4: [4, 3]}
    attn = []
    B1, L1 = x1.size()
    B2, L2 = x2.size()
    for i in range(B1):
      b = []
      for j in range(L2):
        l2 = []
        # x2t = x2[i, j].item()
        x2t = x2[i, j + 1].item() if j + 2 < L2 else 0 # decoder shift
        for k in range(L1):
          x1t = x1[i, k].item()
          # x1t = x1[i, k + 1].item() if k + 1 < L1 else 0 # encoder shift
          x1ts = self.attn_dict.get(x2t, [])
          if x1t in x1ts:
            l2.append(1.)
          else:
            l2.append(0.)
        b.append(l2)
      attn.append(b)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    attn = torch.tensor(attn).to(device)
    attn = F.normalize(attn, p=1, dim=2)  # to [0, 1]
    return attn

from IPython.display import Image
# Image(make_dot(loss).render('loss', format='png'))
from tqdm.notebook import tqdm
from torch import optim
import numpy as np

def plot_attention(a, x1, x2, shift=True, mask=True, 
                   suptitle='Attention', figsize=None, 
                   decoder_x1=None, decoder_x2=None, tight=False,
                   labels=True):
  # %matplotlib inline
  import matplotlib.pyplot as plt
  from mpl_toolkits.axes_grid1 import make_axes_locatable

  b = a.shape[0]
  fig, axs = plt.subplots(1, b, figsize=figsize)
  if not isinstance(axs, np.ndarray):  # if batch size == 1
    axs = [axs]
  fig.suptitle(suptitle)
  for i in range(b):
    ax = axs[i]
    if shift:
      ba, bx1, bx2 = a[i,:-1,1:], x1[i,1:], x2[i,1:]
    else:
      ba, bx1, bx2 = a[i], x1[i], x2[i]
    if mask:
      mask_x1 = np.flatnonzero(bx1)
      mask_x2 = np.flatnonzero(bx2)
      ba, bx1, bx2 =  ba[mask_x2,:][:,mask_x1], bx1[mask_x1], bx2[mask_x2]
    # ax.set_title(f'{i+1}', y=-0.2)
    im = ax.imshow(ba, cmap='gray')
    ax.set_xticks(np.arange(len(bx1)))
    ax.set_yticks(np.arange(len(bx2)))
    if labels:
      ax.set_xlabel('x1')
      ax.set_ylabel('x2')
    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')
    if decoder_x1 is not None:
      bx1 = decoder_x1(bx1)
    if decoder_x2 is not None:
      bx2 = decoder_x2(bx2)
    ax.set_xticklabels(bx1, rotation=90)  # rotation=90
    ax.set_yticklabels(bx2)
    fig.colorbar(im, ax=ax, fraction=0.05, pad=0.05)
  if tight:
    plt.tight_layout()


# torch.backends.cudnn.enabled=False
# torch.backends.cudnn.deterministic=True
# torch.autograd.set_detect_anomaly(True)

# enc_dec_config = {
#   'enc_num_embeddings': 6,
#   'enc_embedding_dim': 2,
#   'enc_padding_idx': 0,
#   'dec_num_embeddings': 6,
#   'dec_embedding_dim': 2,
#   'dec_padding_idx': 0,
#   'rnn_type': 'RNN',
#   'hidden_size': 2,
#   'num_layers': 1,
#   'bidirectional': False,
#   'enc_rnn_dropout': 0,
#   'dec_rnn_dropout': 0,
#   'enc_dropout': 0,
#   'dec_dropout': 0,
#   'attn_type': 'soft_dist',
#   'out_hidden': 16,
#   'pack': True
# }
# # <pad>, <unk>, <go>, <eos>
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# seed = 0
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# model = EncoderDecoder(**enc_dec_config).to(device)
# x1 = torch.tensor([[2, 1, 4, 3, 0], [2, 1, 4, 1, 3]]).to(device)
# x2 = torch.tensor([[2, 4, 1, 5, 3, 0], [2, 4, 5, 1, 1, 3]]).to(device)
# opt = optim.Rprop(model.parameters(), lr=0.01)
# loss_fn = torch.nn.NLLLoss(ignore_index=0)  # 0 is <PAD>
# pbar = tqdm(range(200))
# for i in pbar:
#   opt.zero_grad()
#   t = x2[:,1:].flatten(start_dim=0)
#   y2, attn = model(x1, x2)
#   p = y2[:,:-1,:].flatten(start_dim=0, end_dim=1)
#   loss = loss_fn(p, t)
#   loss.backward()
#   opt.step()
#   with torch.no_grad():
#     idx = torch.nonzero(t).view(-1)
#     acc = torch.sum(torch.argmax(p, 1)[idx] == t[idx]).float()/idx.size()[0]
#     ps = torch.argmax(y2[:,:-1,:], 2).detach().cpu().numpy()
#     ts = x2[:,1:].detach().cpu().numpy()
#     for sp, st in zip(ps, ts):
#       t_set = set(st) - {0, 1, 2, 3}
#       eos = sp[sp == 3][0] if 3 in sp else len(sp)
#       p_eos = sp[:eos]
#       p_set = set(p_eos) - {0, 1, 2, 3}
#       i_set = t_set.intersection(p_set)
#       pre = len(i_set)/len(p_set) if len(p_set) != 0 else 0
#       rec = len(i_set)/len(t_set) if len(t_set) != 0 else 0
#       f1 = 2*pre*rec/(pre + rec) if pre + rec != 0 else 0
#       # print(st, sp, p_eos)
#       # print(t_set, p_set)
#       # print(f'precision: {pre}, recall: {rec}, f1: {f1}')
#     if i % 25 == 0:
#       # print(attn)
#       # print(p)
#       # print(torch.argmax(p, 1))
#       # print(t)
#       if attn is not None:
#          plot_attention(attn.detach().cpu().numpy(), 
#                         x1.detach().cpu().numpy(), 
#                         x2.detach().cpu().numpy(),
#                         labels=True, tight=False,
#                         shift=False, mask=False, figsize=(10, 5))
#   pbar.set_description(f'loss: {loss:.3f}, acc: {acc:.3f}')
#   if acc == 1:
#     break
# # Image(make_dot(attn).render('attn', format='png'))

# Train

## Pretrain

In [0]:
!pip install gputil  # for monitoring
!pip install psutil  # for monitoring
!pip install humanize  # for monitoring

!pip install optuna  # for train

!pip install rouge_score  # for metrics

In [34]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
import subprocess
print(subprocess.getstatusoutput('nvidia-smi')[1])

/bin/sh: 1: nvidia-smi: not found


In [35]:
# Import packages
import os, sys, humanize, psutil, GPUtil

# Define function
def mem_report(verbose=True):
  h = humanize.naturalsize
  cmf = psutil.virtual_memory().available
  cmt = psutil.virtual_memory().total
  if verbose: 
    print(f"CPU mem Free: {h(cmf)} / {h(cmt)}")
  gmfs, gmts, gmus = [], [], []
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    gmfs.append(gpu.memoryFree)
    gmts.append(gpu.memoryTotal)
    gmus.append(gpu.memoryUtil)
    if verbose:
      print(f'GPU {i} mem free: {h(gmfs[-1]*1000000)} / {h(gmts[-1]*1000000)} util: {int(gmus[-1]*100)}%')
  return cmf, gmfs, gmts, gmus
mem_report()

CPU mem Free: 7.5 GB / 13.7 GB


(7462006784, [], [], [])

In [0]:
reduction_map = {
  'sum': torch.sum,
  'mean': torch.mean,
  'none': None
}


class Forced():
  def __init__(self, **kwargs):
    pass

  def __call__(self, model, bx1, bx2):
    by2p, attn = model(bx1, bx2)
    y2 = torch.argmax(by2p[:,:-1,:], axis=2)
    return y2, attn  

class Greedy():
  def __init__(self, sos=2, eos=3, max_len=30, **kwargs):
    self.sos, self.eos, self.max_len = sos, eos, max_len
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


  def __call__(self, model, bx1, bx2=None):
    batch_size = bx1.size(0)  # input batch_size
    bx2 = torch.full((batch_size, 1), self.sos, dtype=bx1.dtype).to(self.device)  # batch with <sos>
    # stop when predictions len > max_len or all have <eos> token
    while bx2.size(1) - 1 < self.max_len and not torch.all(torch.any(bx2 == self.eos, axis=1)):
      by2p, attn = model(bx1, bx2)
      next_bx2 = torch.argmax(by2p[:,-1,:], axis=1, keepdim=True)
      bx2 = torch.cat((bx2, next_bx2), 1)
    return bx2[:,1:], attn  

class Beam():
  def __init__(self, sos=2, eos=3, max_len=30, beam_width=2, beam_depth=2,
               depth_reduction='sum', beam_reduction='none', 
               batch_reduction='none', **kwargs):
    self.sos, self.eos, self.max_len = sos, eos, max_len
    self.beam_width, self.beam_depth = beam_width, beam_depth
    self.depth_reduction = reduction_map[depth_reduction]
    self.beam_reduction = reduction_map[beam_reduction]
    self.batch_reduction = reduction_map[batch_reduction]
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


  def __call__(self, model, bx1, bx2=None):
    batch_size = bx1.size(0)  # input batch_size
    device = torch
    bx2 = torch.full((batch_size, 1), self.sos, dtype=bx1.dtype).to(self.device)  # batch with <sos>
    # stop when len decoder output > max_len or all decoder outputs have <eos> token
    while bx2.size(1) - 1 < self.max_len and not torch.all(torch.any(bx2 == self.eos, axis=1)):
      beam_scores = torch.empty((batch_size, 0)).to(self.device)  # scores for each beam
      bx1t = bx1.clone()
      for i in range(self.beam_depth):                                           
        new_batch_size = batch_size*self.beam_width**(i+1)
        by2p, attn = model(bx1t, bx2)  # predict
        if i > 0 and self.beam_reduction is not None:  # beam reduction
          prev_prev_batch_size = batch_size*self.beam_width**(i-1)
          by2p = by2p.view(prev_prev_batch_size, self.beam_width, by2p.size(1), -1)  # split predictions into batches
          by2p = self.beam_reduction(by2p, axis=1) # cumulative beams predictions
          by2p = by2p.repeat(self.beam_width, 1, 1)  # return to prev batch size
        if self.batch_reduction is not None:  # batch reduction
          prev_batch_size = batch_size*self.beam_width**(i)
          by2p = self.batch_reduction(by2p, axis=0) # cumulative batch predictions
          next_by2p, next_bx2 = torch.topk(by2p[-1,:], self.beam_width)  # beams to top k last predictions
          next_bx2 = next_bx2.repeat(prev_batch_size, 1)  # return to prev batch size
          next_by2p = next_by2p.repeat(prev_batch_size, 1)  # return to prev batch size
        else:
          next_by2p, next_bx2 = torch.topk(by2p[:,-1,:], self.beam_width)  # beams to top k last predictions
        next_bx2 = next_bx2.view(new_batch_size, 1)  # new beams
        next_by2p = next_by2p.view(new_batch_size, 1)  # new scores
        beam_scores = torch.repeat_interleave(beam_scores, self.beam_width, 0)  # increase batch for new scores
        bx2 = torch.repeat_interleave(bx2, self.beam_width, 0)  # increase batch for new beams
        bx2 = torch.cat((bx2, next_bx2), 1)  # add beams
        beam_scores = torch.cat((beam_scores, next_by2p), 1)  # add beams scores
        bx1t = torch.repeat_interleave(bx1t, self.beam_width, 0)  # increase batch for new beams
      beam_scores = self.depth_reduction(beam_scores, axis=1) # cumulative beams scores
      if self.batch_reduction is None:
        beam_scores = beam_scores.view(batch_size, -1)  # split scores into batches
        best_beams = torch.argmax(beam_scores, axis=1, keepdim=True)  # best beams
        bx2 = bx2.view(batch_size, self.beam_width**self.beam_depth, -1)  # split beams into batches
        # XXX its fucking magic... (return to input batch_size)
        best_beams = best_beams.unsqueeze(2).expand(best_beams.size(0), 
                                                    best_beams.size(1),
                                                    bx2.size(2))
        bx2 = torch.gather(bx2, 1, best_beams)
        bx2 = bx2.view(batch_size, -1)
      else:
        best_beams = torch.argmax(beam_scores, axis=0, keepdim=True)  # best beams
        bx2 = bx2[best_beams]  # best beam of all batches
        bx2 = bx2.repeat(batch_size, 1)  # return to input batch size
    return bx2[:,1:], attn

class BeamBkw():
  def __init__(self, sos=2, eos=3, max_len=30, beam_width=2, beam_depth=2,
               depth_reduction='sum', beam_reduction='none', 
               batch_reduction='none', bkw_beam_width=2, **kwargs):
    self.sos, self.eos, self.max_len = sos, eos, max_len
    self.beam_width, self.beam_depth = beam_width, beam_depth
    self.bkw_beam_width = bkw_beam_width
    self.depth_reduction = reduction_map[depth_reduction]
    self.beam_reduction = reduction_map[beam_reduction]
    self.batch_reduction = reduction_map[batch_reduction]
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if self.bkw_beam_width > self.beam_width**self.beam_depth:
      print(f'Warning! Backward beam width {self.bkw_beam_width} greater\
      than beam_width**beam_depth {self.beam_width**self.beam_depth}') 
      print(f'setting backword beam width to {self.beam_width**self.beam_depth}')
      self.bkw_beam_width = self.beam_width**self.beam_depth

  def __call__(self, model, bx1, bx2=None):
    batch_size = bx1.size(0)  # input batch_size
    bx2 = torch.full((batch_size, 1), self.sos, dtype=bx1.dtype).to(self.device)  # batch with <sos>
    # stop when len decoder output > max_len or all decoder outputs have <eos> token
    while bx2.size(1) - 1 < self.max_len and not torch.all(torch.any(bx2 == self.eos, axis=1)):
      fwd_scores = torch.empty((batch_size, 0)).to(self.device)  # scores for each beam
      fwd_bx2 = bx2.clone()
      bx1t = bx1.clone()
      for i in range(self.beam_depth):
        new_batch_size = batch_size*self.beam_width**(i+1)
        by2p, attn = model(bx1t, fwd_bx2)  # predict
        if i > 0 and self.beam_reduction is not None:  # beam reduction
          prev_prev_batch_size = batch_size*self.beam_width**(i-1)
          by2p = by2p.view(prev_prev_batch_size, self.beam_width, by2p.size(1), -1)  # split predictions into batches
          by2p = self.beam_reduction(by2p, axis=1) # cumulative beams predictions
          by2p = by2p.repeat(self.beam_width, 1, 1)  # return to prev batch size
        if self.batch_reduction is not None:
          prev_batch_size = batch_size*self.beam_width**(i)
          by2p = self.batch_reduction(by2p, axis=0) # cumulative batch predictions
          next_by2p, next_bx2 = torch.topk(by2p[-1,:], self.beam_width)  # beams to top k last predictions
          next_bx2 = next_bx2.repeat(prev_batch_size, 1)  # return to prev batch size
          next_by2p = next_by2p.repeat(prev_batch_size, 1)  # return to prev batch size        
        else:
          next_by2p, next_bx2 = torch.topk(by2p[:,-1,:], self.beam_width)  # beams to top k last predictions        
        next_bx2 = next_bx2.view(new_batch_size, 1)  # new beams
        next_by2p = next_by2p.view(new_batch_size, 1)  # new scores
        fwd_scores = torch.repeat_interleave(fwd_scores, self.beam_width, 0)  # increase batch for new scores
        fwd_bx2 = torch.repeat_interleave(fwd_bx2, self.beam_width, 0)  # increase batch for new beams
        fwd_bx2 = torch.cat((fwd_bx2, next_bx2), 1)  # add beams
        fwd_scores = torch.cat((fwd_scores, next_by2p), 1)  # add beams scores
        bx1t = torch.repeat_interleave(bx1t, self.beam_width, 0)  # increase batch for new beams
      if self.batch_reduction is None:
        fwd_scores = fwd_scores.view(batch_size, self.beam_width**self.beam_depth, -1)  # split scores into batches
        fwd_bx2 = fwd_bx2.view(batch_size, self.beam_width**self.beam_depth, -1)  # split beams into batches
        bkw_bx2 = torch.empty((batch_size, self.beam_width**self.beam_depth, 0), dtype=bx1.dtype)  # scores for each beam
        bkw_scores = torch.empty((batch_size, self.beam_width**self.beam_depth, 0))  # scores for each beam
        mask = torch.full((fwd_bx2.size(0), fwd_bx2.size(1), 1), False, dtype=torch.bool)
        n_beams_per_batch = self.beam_width**self.beam_depth
        for i in range(fwd_scores.size(2)):
          old_n_beams_per_batch = n_beams_per_batch*self.bkw_beam_width**(i)
          new_n_beams_per_batch = n_beams_per_batch*self.bkw_beam_width**(i+1)
          cur_values = fwd_bx2[:,:,-i-1:-i if i > 0 else None]
          cur_values = torch.repeat_interleave(cur_values, self.bkw_beam_width**i, 1)
          cur_scores = fwd_scores[:,:,-i-1:-i if i > 0 else None]
          cur_scores = torch.repeat_interleave(cur_scores, self.bkw_beam_width**i, 1)
          cur_scores[mask] = float('-inf')  # mask from prev step
          best_scores, best_beams = torch.topk(cur_scores, self.bkw_beam_width, 1)  # best beams
          best_values = torch.gather(cur_values, 1, best_beams).view(batch_size, -1)
          best_values = best_values.view(best_values.size(0), self.bkw_beam_width, -1)
          best_values = torch.repeat_interleave(best_values, old_n_beams_per_batch, 1)
          cur_values = cur_values.repeat(1, self.bkw_beam_width, 1)
          mask = cur_values != best_values
          bkw_bx2 = bkw_bx2.repeat(1, self.bkw_beam_width, 1)  # increase batch for new beams
          bkw_bx2 = torch.cat((best_values, bkw_bx2), 2)
          best_scores = torch.repeat_interleave(best_scores, old_n_beams_per_batch, 1)
          bkw_scores = bkw_scores.repeat(1, self.bkw_beam_width, 1)  # increase batch for new beams
          bkw_scores = torch.cat((best_scores, bkw_scores), 2)
        beam_scores = self.depth_reduction(bkw_scores, axis=2) # cumulative beams scores
        best_beams = torch.argmax(beam_scores, axis=1, keepdim=True)  # best beams
        best_beams = best_beams.unsqueeze(2).expand(best_beams.size(0), best_beams.size(1), bkw_bx2.size(2))
        bkw_bx2 = torch.gather(bkw_bx2, 1, best_beams) # best beam of all batches
        bkw_bx2 = bkw_bx2.view(batch_size, -1)
        bx2 = torch.cat((bx2, bkw_bx2), 1)  # <sos> + bkw_bx2
      else:  # batch reduction
        bkw_bx2 = torch.empty((new_batch_size, 0), dtype=bx1.dtype)  # scores for each beam
        bkw_scores = torch.empty((new_batch_size, 0))  # scores for each beam
        mask = torch.full((new_batch_size, 1), False, dtype=torch.bool)
        fwd_batch_size = new_batch_size
        for i in range(fwd_scores.size(1)):
          prev_batch_size = fwd_batch_size*self.bkw_beam_width**(i)
          new_batch_size = fwd_batch_size*self.bkw_beam_width**(i+1)
          cur_values = fwd_bx2[:,-i-1:-i if i > 0 else None]
          cur_values = torch.repeat_interleave(cur_values, self.bkw_beam_width**i, 0)
          cur_scores = fwd_scores[:,-i-1:-i if i > 0 else None]
          cur_scores = torch.repeat_interleave(cur_scores, self.bkw_beam_width**i, 0)
          cur_scores[mask] = float('-inf')  # mask from prev step
          best_scores, best_beams = torch.topk(cur_scores, self.bkw_beam_width, 0)  # best beams
          best_values = cur_values[best_beams]
          mask = cur_values != best_values
          mask = mask.view(new_batch_size, -1)
          best_values = best_values.view(self.bkw_beam_width, -1)
          best_values = torch.repeat_interleave(best_values, prev_batch_size, 0)
          bkw_bx2 = bkw_bx2.repeat(self.bkw_beam_width, 1)  # increase batch for new beams
          bkw_bx2 = torch.cat((best_values, bkw_bx2), 1)
          best_scores = torch.repeat_interleave(best_scores, prev_batch_size, 0)
          bkw_scores = bkw_scores.repeat(self.bkw_beam_width, 1)  # increase batch for new beams
          bkw_scores = torch.cat((best_scores, bkw_scores), 1)
        # bkw batch reduction
        bkw_scores = self.depth_reduction(bkw_scores, axis=1) # cumulative beams scores
        best_beams = torch.argmax(bkw_scores, axis=0, keepdim=True)  # best beams
        bkw_bx2 = bkw_bx2[best_beams]  # best beam of all batches
        bkw_bx2 = bkw_bx2.repeat(batch_size, 1)  # return to input batch size
        bx2 = torch.cat((bx2, bkw_bx2), 1)  # <sos> + bkw_bx2
    return bx2[:,1:], attn

In [75]:
from rouge_score import rouge_scorer

def metrics(bx2, by2, eos=3):
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])
  scores = {}
  ps = by2.detach().cpu().numpy()
  ts = bx2.detach().cpu().numpy()
  for sp, st in zip(ps, ts):
    # target to set
    st_eos_i = np.where(st==eos)[0][0] if eos in st else len(st)
    st_eos = st[:st_eos_i]
    t_set = set(st_eos) - {0, 1, 2, 3}
    # prediction to set
    sp_eos_i = np.where(sp==eos)[0][0] if eos in sp else len(sp)
    sp_eos = sp[:sp_eos_i]
    p_set = set(sp_eos) - {0, 1, 2, 3}
    # metrics
    i_set = t_set.intersection(p_set)
    pre = len(i_set)/len(p_set) if len(p_set) != 0 else 0
    rec = len(i_set)/len(t_set) if len(t_set) != 0 else 0
    f1 = 2*pre*rec/(pre + rec) if pre + rec != 0 else 0
    rl = len(sp_eos)/len(st_eos) if len(st_eos) != 0 else 0
    # to string
    t_str = ' '.join(map(str, st_eos))
    p_str = ' '.join(map(str, sp_eos))
    rs = scorer.score(t_str, p_str)
    # print(f'st: {st}, sp: {sp}')
    # print(f'eos in st?: {eos in st}, sp_eos_i: {st_eos_i}')
    # print(f'eos in sp?: {eos in sp}, sp_eos_i: {sp_eos_i}')
    # print(f'st_eos: {st_eos}, sp_eos: {sp_eos}')
    # print(f't_set: {t_set}, p_set: {p_set}')
    # print(f't_str: "{t_str}", p_str: "{p_str}"')
    # print(f'precision: {pre}, recall: {rec}, f1: {f1}. rl: {rl}\n')
    scores.setdefault('set_precision', []).append(pre)
    scores.setdefault('set_recall', []).append(rec)
    scores.setdefault('set_f1', []).append(f1)
    scores.setdefault('relen', []).append(rl)
    for k, v in rs.items():
      scores.setdefault(f'{k}_precision', []).append(v.precision)
      scores.setdefault(f'{k}_recall', []).append(v.recall)
      scores.setdefault(f'{k}_f1', []).append(v.fmeasure)
  return scores

x2 = torch.tensor([[1, 5, 3, 4, 4, 3, 3, 3, 3], 
                   [5, 3, 0, 4, 3, 0, 0, 0, 0]])
y2 = torch.tensor([[3, 1, 4, 3, 4, 6, 7, 4], 
                   [0, 5, 4, 3, 3, 0, 0, 3]])
metrics(x2, y2)

{'relen': [0.0, 3.0],
 'rouge1_f1': [0.0, 0.5],
 'rouge1_precision': [0.0, 0.3333333333333333],
 'rouge1_recall': [0.0, 1.0],
 'rouge2_f1': [0.0, 0.0],
 'rouge2_precision': [0.0, 0.0],
 'rouge2_recall': [0.0, 0.0],
 'rougeL_f1': [0, 0.5],
 'rougeL_precision': [0, 0.3333333333333333],
 'rougeL_recall': [0, 1.0],
 'set_f1': [0, 0.6666666666666666],
 'set_precision': [0, 0.5],
 'set_recall': [0.0, 1.0]}

In [0]:
import humanize
import psutil
import GPUtil

def run(train, device, model, opt, loss_fn, dl, ds, print_triples=1, 
        plot_attn=False, inference=None, metric=False, pin_memory=False,
        max_samples=None):
  scores = {}
  samples_cnt = 0
  pbar = tqdm(dl, total=max_samples)
  if train:
    model.train()
    for x1, x2 in pbar:
      samples_cnt += 1
      opt.zero_grad()
      x1 = x1.to(device, non_blocking=pin_memory)
      x2 = x2.to(device, non_blocking=pin_memory)
      t = x2[:,1:].flatten(0)
      y2p, attn = model(x1, x2)
      p = y2p[:,:-1,:].flatten(0, 1)
      loss = loss_fn(p, t)
      loss.backward()
      opt.step()
      scores.setdefault('loss', []).extend(loss.item())
      y2 = None
      if metric:
        y2 = torch.argmax(y2p[:,:-1,:], 2)
        # idx = torch.nonzero(t).view(-1)
        # acc = torch.sum(y2.view(-1)[idx] == t[idx]).float()/idx.size()[0]
        # accs.append(acc.item())
        bs = metrics(x2[:,1:], y2)  # batch scores
        r1f = sum(bs['rouge1_f1'])/len(bs['rouge1_f1'])
        r2f = sum(bs['rouge2_f1'])/len(bs['rouge2_f1'])
        rlf = sum(bs['rougeL_f1'])/len(bs['rougeL_f1'])
        sf = sum(bs['set_f1'])/len(bs['set_f1'])
        rl = sum(bs['relen'])/len(bs['relen'])
        for score, values in bs.items():
          scores.setdefault(score, []).extend(values)
        cmt = psutil.virtual_memory().total
        cma = psutil.virtual_memory().available
        cmu = (cmt-cma)/cmt
        gmu = GPUtil.getGPUs()[0].memoryUtil if len(GPUtil.getGPUs()) > 0 else 0
        pbar.set_description(f'train {int(cmu*100)}%/{int(gmu*100)}% \
          l: {loss.item():.3}, \
          sf: {int(sf*100)}, r1f: {int(r1f*100)}, r2f: {int(r2f*100)}, \
          rlf: {int(rlf*100)}, rl: {int(rl*100)}')
      else:
        pbar.set_description(f'loss: {loss.item():.3}')
      if max_samples is not None:
          if samples_cnt == max_samples:
            break
  else:
    model.eval()
    with torch.no_grad():
      for x1, x2 in pbar:
        samples_cnt += 1
        x1 = x1.to(device, non_blocking=pin_memory)
        x2 = x2.to(device, non_blocking=pin_memory)
        if inference is not None:
          y2, attn = inference(model, x1, x2)
        else:  # forced with loss (e.g. for training)
          t = x2[:,1:].flatten(0)
          y2p, attn = model(x1, x2)
          p = y2p[:,:-1,:].flatten(0, 1)
          loss = loss_fn(p, t)
          y2 = torch.argmax(y2p[:,:-1,:], 2)
          scores.setdefault('loss', []).extend(loss.item())
          pbar.set_description(f'loss: {loss.item():.3}')
        if metric:
          bs = metrics(x2[:,1:], y2)  # batch scores
          for score, values in bs.items():
            scores.setdefault(score, []).extend(values)
        if max_samples is not None:
          if samples_cnt == max_samples:
            break
          # r1f = sum(bs['rouge1_f1'])/len(bs['rouge1_f1'])
          # r2f = sum(bs['rouge2_f1'])/len(bs['rouge2_f1'])
          # rlf = sum(bs['rougeL_f1'])/len(bs['rougeL_f1'])
          # sf = sum(bs['set_f1'])/len(bs['set_f1'])
          # rl = sum(bs['relen'])/len(bs['relen'])
          # cmt = psutil.virtual_memory().total
          # cma = psutil.virtual_memory().available
          # cmu = (cmt-cma)/cmt
          # gmu = GPUtil.getGPUs()[0].memoryUtil if len(GPUtil.getGPUs()) > 0 else 0
          # pbar.set_description(f'eval {int(cmu*100)}%/{int(gmu*100)}% \
          #   sf: {int(sf*100)}, r1f: {int(r1f*100)}, r2f: {int(r2f*100)}, \
          #   rlf: {int(rlf*100)}, rl: {int(rl*100)}')
          # pbar.set_description(f'gmu: {int(gmu*100)}%')
  if print_triples:
    if y2 is None:
      y2 = torch.argmax(y2p[:,:-1,:], 2)
    dy2 = map(ds.decode, y2.detach().cpu().numpy())
    dx2 = map(ds.decode, x2[:,1:].detach().cpu().numpy())
    dx1 = map(ds.decode, x1[:,1:].detach().cpu().numpy())
    triples = list(zip(dx1, dx2, dy2))[:print_triples]
    for triple in triples:
      print('')
      print(triple[0])
      print(triple[1])
      print(triple[2])
  if plot_attn:
    plot_attention(attn.detach().cpu().numpy()[:1], 
                   x1.detach().cpu().numpy()[:1], 
                   x2.detach().cpu().numpy()[:1],
                   decoder_x1 = ds.decode,
                   decoder_x2 = ds.decode,
                   figsize=(10, 10),
                   shift=True,
                   mask=True,
                   suptitle=None)
  return scores

## Main

In [0]:
import torch
import optuna
from tqdm.notebook import tqdm
from pprint import pprint
import random
import numpy as np
from shutil import copyfile
import GPUtil
from torch import optim
from torchsummary import summary

def objective(trial):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  trial.set_user_attr('device', str(device))
  if device.type == "cuda":
    print(f'gmu: {"% ".join(str(int(x.memoryUtil*100)) for x in GPUtil.getGPUs())}%')
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    print(torch.cuda.memory_stats(device=device))
    print(torch.cuda.memory_summary(device=device))
    print(f'gmu: {"% ".join(str(int(x.memoryUtil*100)) for x in GPUtil.getGPUs())}%')
  opt_fn_map = {
    'SGD': optim.SGD,
    'Adam': optim.Adam,
    'Adagrad': optim.Adagrad,
    'ASGD': optim.ASGD,
    'Adamax': optim.Adamax,
    'SparseAdam': optim.SparseAdam,
    'AdamW': optim.AdamW,
    'Adadelta': optim.Adadelta,
    'LBFGS': optim.LBFGS,
    'RMSprop': optim.RMSprop,
    'Rprop': optim.Rprop
  }
  eval_fn_map = {
      'Beam': Beam,
      'BeamBkw': BeamBkw,
      'Greedy': Greedy,
      'Forced': Forced
  }
  # fn = 'model.pth'
  fn = f'model_{trial.study.study_name}_{trial.number}.pth'
  print(fn)
  # checkpoint = None
  checkpoint = torch.load('/content/drive/My Drive/model_ria_base.pth',
                          map_location=device)
  train = False  # ?
  # train_config = checkpoint['train_config']
  train_config = {
    'seed': trial.suggest_int('seed', 0, 0),
    'n_epoches': trial.suggest_int('n_epoches', 1, 1),
    'lr': trial.suggest_loguniform('lr', 1e-3, 1e-3),
      # 'opt_fn': trial.suggest_categorical('opt_fn', ['SGD', 'Adam', 'Adagrad', 
      #                                           'ASGD', 'Adamax',
      #                                           'AdamW', 'Adadelta',
      #                                           'RMSprop', 'Rprop']),
    'opt_fn': trial.suggest_categorical('opt_fn', ['Adam']),
    'batch_size': trial.suggest_int('batch_size', 1, 1),
    'weight_decay': None,
    'pin_memory': True,
    # 'weight_decay': trial.suggest_loguniform('weight_decay', 1e-6, 1e-6),
    'n_x1': trial.suggest_int('n_x1', 1, 1)
  }
  eval_config = {
      'eval_fn': trial.suggest_categorical('eval_fn', ['Beam', 'BeamBkw']),
      # 'eval_fn': 'BeamBkw',
      'max_samples': 100}
  if eval_config['eval_fn'] in ['Beam', 'BeamBkw']:
    eval_config['beam_width'] = trial.suggest_int('beam_width', 1, 20)
    eval_config['beam_depth'] = trial.suggest_int('beam_depth', 2, 3)
    # eval_config['beam_reduction'] = trial.suggest_categorical('beam_reduction', ['sum', 'mean', 'none']),
    eval_config['beam_reduction'] = 'none'
    eval_config['batch_reduction'] = 'none'
    eval_config['depth_reduction'] = trial.suggest_categorical('depth_reduction', ['sum', 'mean'])
    if eval_config['eval_fn'] == 'BeamBkw':
      eval_config['bkw_beam_width'] = trial.suggest_int('bkw_beam_width', 1, 20)
  if checkpoint is None:
    model_config = {
      'enc_num_embeddings': len(ds.stoi),
      'enc_embedding_dim': trial.suggest_int('enc_embedding_dim', 300, 300),
      'enc_padding_idx': 0,
      'dec_num_embeddings': len(ds.stoi),
      'dec_embedding_dim': trial.suggest_int('dec_embedding_dim', 300, 300),
      'dec_padding_idx': 0,
      'rnn_type': trial.suggest_categorical('rnn_type', ['RNN']),
      # 'rnn_type': trial.suggest_categorical('rnn_type', ['RNN']),
      'hidden_size': trial.suggest_int('hidden_size', 300, 300),
      'num_layers': trial.suggest_int('num_layers', 1, 1),
      'bidirectional': trial.suggest_categorical('bidirectional', [False]),
      # 'bidirectional': trial.suggest_categorical('bidirectional', [True]),
      'enc_dropout': trial.suggest_uniform('enc_dropout', 0.0, 0.0),
      'dec_dropout': trial.suggest_uniform('dec_dropout', 0.0, 0.0),
      'enc_rnn_dropout': trial.suggest_uniform('enc_rnn_dropout', 0.0, 0.0),
      'dec_rnn_dropout': trial.suggest_uniform('dec_rnn_dropout', 0.0, 0.0),
      'attn_type': trial.suggest_categorical('attn_type', ['soft_dist']),
      # 'attn_type': trial.suggest_categorical('attn_type', ['dot', 'dist', 'soft_dot', 'soft_dist', 'cos', 'soft_cos', 'none']),
      'out_hidden': trial.suggest_int('out_hidden', 600, 600),
      'pack': trial.suggest_categorical('pack', [True]),
    }
  else:
    model_config = checkpoint['model_config']
  pprint(train_config)
  pprint(model_config)
  pprint(eval_config)
  seed = train_config['seed']
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  random.seed(seed)
  np.random.seed(seed)
  opt_fn = train_config['opt_fn']
  lr = train_config['lr']
  n_epoches = train_config['n_epoches']
  batch_size = train_config['batch_size']
  weight_decay = train_config['weight_decay']
  pin_memory = train_config['pin_memory']
  trial.set_user_attr('pin_memory', train_config['pin_memory'])
  trial.set_user_attr('max_samples', eval_config['max_samples'])
  model = EncoderDecoder(**model_config).to(device)
  if checkpoint is not None:
    model.load_state_dict(checkpoint['model_state_dict'])
  pprint(model)
  n_params = sum(x.numel() for x in model.parameters() if x.requires_grad)
  trial.set_user_attr('n_params', n_params)
  if opt_fn not in ['Rprop'] and weight_decay is not None:
    opt = opt_fn_map[opt_fn](model.parameters(), lr=lr, 
                             weight_decay=weight_decay)
  else:
    opt = opt_fn_map[opt_fn](model.parameters(), lr=lr)
  loss_fn = torch.nn.NLLLoss(ignore_index=0)
  train_len = int(0.7*len(ds))
  test_len = int(0.2*len(ds))
  val_len = len(ds) - train_len - test_len
  lens = [train_len, test_len, val_len]
  trial.set_user_attr('n_samples', len(ds))
  trial.set_user_attr('n_samples_train', train_len)
  trial.set_user_attr('n_samples_test', test_len)
  trial.set_user_attr('n_samples_val', val_len)
  train_ds, test_ds, val_ds = random_split(ds, lens)
  train_dl = DataLoader(train_ds, batch_size=batch_size, num_workers=1, 
                        shuffle=True, drop_last=False,
                        collate_fn=Collate(train_config['n_x1']),
                        pin_memory=pin_memory)
  test_dl = DataLoader(test_ds, batch_size=batch_size, num_workers=1, 
                       shuffle=False, drop_last=False,
                       collate_fn=Collate(train_config['n_x1']),
                       pin_memory=pin_memory)
  val_dl = DataLoader(val_ds, batch_size=batch_size, num_workers=1, 
                      shuffle=False, drop_last=False,
                      collate_fn=Collate(train_config['n_x1']),
                      pin_memory=pin_memory)
  pprint(trial.params)
  pprint(trial.user_attrs)
  if train:
    pbar_epoch = tqdm(range(n_epoches))
    for i in tqdm(pbar_epoch):
      train_scores = run(True, device, model, opt, loss_fn, train_dl, ds, 
                          print_triples=3, 
                          metric=False,
                          pin_memory=pin_memory,
                          plot_attn=True if model_config['attn_type'] != 'none' else False)
      train_loss = sum(train_scores['loss'])/len(train_scores['loss'])
      trial.report(train_loss, step=i+1)
      # validation
      # eval_fn = eval_fn_map[eval_config['eval_fn']](**eval_config)
      val_scores = run(False, device, model, opt, loss_fn, val_dl, ds,  
                       print_triples=3, 
                       inference=None,
                       metric=False,
                       pin_memory=pin_memory,
                       plot_attn=False)
      val_loss = sum(val_scores['loss'])/len(val_scores['loss'])
      # print(f'{type(eval_fn).__name__} p: {val_pre:.3f}, r: {val_rec:.3f}, f1: {val_f1:.3f}, rl: {val_rl:.3f}')
      pbar_epoch.set_description(f'epoch: {i+1}, tloss: {train_loss:.3}, vloss: {val_loss:.3}')
      # pbar_epoch.set_description(f'l: {loss:.2f}/{val_loss:.2f} \
      # a: {int(acc*100)}/{int(val_acc*100)} p: {int(pre*100)}/{int(val_pre*100)} \
      # r: {int(rec*100)}/{int(val_rec*100)} f1: {int(f1*100)}/{int(val_f1*100)}\
      # rl: {int(rl*100)}/{int(val_rl*100)}')
      # torch.save({'epoch': i,
      #             'model_state_dict': model.state_dict(),
      #             'optimizer_state_dict': opt.state_dict(),
      #             'loss': loss,
      #             'model_config': model_config,
      #             'train_config': train_config,
      #             'eval_config': eval_config},
      #              fn)
      # try:
      #   copyfile(fn, '/content/drive/My Drive/' + fn)
      # except Exception as e:
      #   print(e)
  # test
  eval_fn = eval_fn_map[eval_config['eval_fn']](**eval_config)
  test_scores = run(False, device, model, opt, loss_fn, test_dl, ds, 
                    print_triples=10, 
                    inference=eval_fn,
                    metric=True,
                    pin_memory=pin_memory,
                    max_samples=eval_config['max_samples'],
                    plot_attn=False)
  scores = test_scores
  test_scores = {}
  for score, values in scores.items():
    test_scores[score] = sum(values)/len(values)
    trial.set_user_attr(score, test_scores[score])
  pprint(test_scores)
  return loss if train else test_scores['rouge1_f1']

import os
fn = 'optuna_20.db'
if not os.path.exists(fn):
  copyfile('/content/drive/My Drive/' + fn, fn)
study = optuna.create_study(study_name='beam vs beam_bkw 2', 
                            direction='maximize', 
                            storage=f'sqlite:///{fn}', 
                            sampler=optuna.samplers.RandomSampler(0),  # optuna.samplers.GridSampler()
                            load_if_exists=True)

def dump_optuna(study, frozen_trial):
  fn = 'optuna_20.db'
  print(f'saving optuna {fn}')
  try:
    copyfile(fn, '/content/drive/My Drive/' + fn)
  except Exception as e:
    print(e)
  print('done')

study.optimize(objective, n_trials=10, callbacks=[dump_optuna])

[I 2020-05-22 11:49:26,976] Using an existing study with name 'beam vs beam_bkw 2' instead of creating a new one.


model_beam vs beam_bkw 2_15.pth
{'batch_size': 1,
 'lr': 0.001,
 'n_epoches': 1,
 'n_x1': 1,
 'opt_fn': 'Adam',
 'pin_memory': True,
 'seed': 0,
 'weight_decay': None}
{'attn_type': 'soft_dist',
 'bidirectional': False,
 'dec_dropout': 0.0,
 'dec_embedding_dim': 300,
 'dec_num_embeddings': 50004,
 'dec_padding_idx': 0,
 'dec_rnn_dropout': 0.0,
 'enc_dropout': 0.0,
 'enc_embedding_dim': 300,
 'enc_num_embeddings': 50004,
 'enc_padding_idx': 0,
 'enc_rnn_dropout': 0.0,
 'hidden_size': 200,
 'num_layers': 1,
 'out_hidden': 600,
 'pack': True,
 'rnn_type': 'RNN'}
{'batch_reduction': 'none',
 'beam_depth': 3,
 'beam_reduction': 'none',
 'beam_width': 16,
 'depth_reduction': 'sum',
 'eval_fn': 'Beam',
 'max_samples': 100}
EncoderDecoder(
  (encoder): EncoderRNN(
    (embedding): Embedding(50004, 300, padding_idx=0)
    (rnn): RNN(300, 200, batch_first=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (decoder): AttentionDecoderRNN(
    (embedding): Embedding(50004, 300, padding_idx=0)

In [0]:
import gc
gc.collect()
# try:
#   copyfile(fn, '/content/drive/My Drive/' + fn)
# except Exception as e:
#   print(e)

In [0]:
# Greedy 200k
# {'relen': 0.9892607721833844,
#  'rouge1_f1': 0.38423491185005065,
#  'rouge1_precision': 0.4026889198348889,
#  'rouge1_recall': 0.37827062666600875,
#  'rouge2_f1': 0.16623936163781577,
#  'rouge2_precision': 0.17458141088353102,
#  'rouge2_recall': 0.16407297971294255,
#  'rougeL_f1': 0.3567234511838257,
#  'rougeL_precision': 0.3739773768993677,
#  'rougeL_recall': 0.35121314215560523,
#  'set_f1': 0.39064669071317015,
#  'set_precision': 0.4199063773953252,
#  'set_recall': 0.3771313731307635}

# Beam 100 w10 d3
# {'relen': 1.0699595543345541,
#  'rouge1_f1': 0.3837477940247945,
#  'rouge1_precision': 0.3846962065712068,
#  'rouge1_recall': 0.39880189255189263,
#  'rouge2_f1': 0.17458943300289448,
#  'rouge2_precision': 0.17466804029304026,
#  'rouge2_recall': 0.18444368131868127,
#  'rougeL_f1': 0.3616800757325285,
#  'rougeL_precision': 0.3621076839826841,
#  'rougeL_recall': 0.37483211233211233,
#  'set_f1': 0.3891797101214522,
#  'set_precision': 0.394974816849817,
#  'set_recall': 0.39936278998779007}

# Beam 1000 w10 d3
# {'relen': 1.0291088694055617,
#  'rouge1_f1': 0.3972183821690979,
#  'rouge1_precision': 0.4098572228679235,
#  'rouge1_recall': 0.3952877861438179,
#  'rouge2_f1': 0.1823947491768629,
#  'rouge2_precision': 0.18893096473349352,
#  'rouge2_recall': 0.18151645695419977,
#  'rougeL_f1': 0.3727206457621715,
#  'rougeL_precision': 0.38427433731422067,
#  'rougeL_recall': 0.37105325770209074,
#  'set_f1': 0.4033988435801159,
#  'set_precision': 0.425185674139954,
#  'set_recall': 0.39412896334102604}


# Greedy
# {'relen': 1.0421636615811374,
#  'rouge1_f1': 0.36153514845089574,
#  'rouge1_precision': 0.37380251894815014,
#  'rouge1_recall': 0.37073509015256595,
#  'rouge2_f1': 0.14549261418193457,
#  'rouge2_precision': 0.14574664817383262,
#  'rouge2_recall': 0.15383726306056406,
#  'rougeL_f1': 0.3315514590011869,
#  'rougeL_precision': 0.34281161125821313,
#  'rougeL_recall': 0.33961704422869465,
#  'set_f1': 0.3667970766996887,
#  'set_precision': 0.3861843119124675,
#  'set_recall': 0.3698566805362921}


# {'relen': 1.0187751963773364,
#  'rouge1_f1': 0.3902193479165969,
#  'rouge1_precision': 0.4062600013475498,
#  'rouge1_recall': 0.3868606506981994,
#  'rouge2_f1': 0.16960439274485042,
#  'rouge2_precision': 0.1767683435241799,
#  'rouge2_recall': 0.16903459974957996,
#  'rougeL_f1': 0.3644276370525009,
#  'rougeL_precision': 0.379012224805026,
#  'rougeL_recall': 0.3616888731674725,
#  'set_f1': 0.39688228655519225,
#  'set_precision': 0.4242750602186396,
#  'set_recall': 0.3849927148078899}


# Beam
# {'relen': 1.0449375866851591,
#  'rouge1_f1': 0.3643022291764363,
#  'rouge1_precision': 0.36877477969711003,
#  'rouge1_recall': 0.3772152874094622,
#  'rouge2_f1': 0.16458721968430698,
#  'rouge2_precision': 0.1653374942209893,
#  'rouge2_recall': 0.17312760055478496,
#  'rougeL_f1': 0.33678510514303583,
#  'rougeL_precision': 0.3409238011179759,
#  'rougeL_recall': 0.3471759901371551,
#  'set_f1': 0.3695013771684244,
#  'set_precision': 0.37901448605332116,
#  'set_recall': 0.3778625366004007}

# 100
#       Forced p: 0.480, r: 0.388, f1: 0.423, rl: 0.968
#       Greedy p: 0.386, r: 0.370, f1: 0.367, rl: 1.042
# Beam w2d2sum p: 0.387, r: 0.391, f1: 0.381, rl: 1.088
# Beam w3d3sum p: 0.387, r: 0.393, f1: 0.381, rl: 1.074
# Beam w3d5sum p: 0.380, r: 0.383, f1: 0.374, rl: 1.073
#         Beam p: 0.380, r: 0.388, f1: 0.376, rl: 1.078
#      BeamBkw p: 0.416, r: 0.308, f1: 0.346, rl: 1.095
#      BeamBkw p: 0.442, r: 0.362, f1: 0.384, rl: 1.168
#      BeamBkw p: 0.423, r: 0.348, f1: 0.375, rl: 1.078
# 1000
#         Beam p: 0.421, r: 0.386, f1: 0.397, rl: 1.017 
#      BeamBkw p: 0.433, r: 0.339, f1: 0.373, rl: 1.050
#      BeamBkw p: 0.435, r: 0.339, f1: 0.374, rl: 1.055
#       Greedy p: 0.424, r: 0.385, f1: 0.397, rl: 1.019

# try:
# except KeyboardInterrupt:
#   print(e)
# except Exception as e:
#   print(e)
# finally:
#   torch.save({'epoch': i,
#               'model_state_dict': model.state_dict(),
#               'optimizer_state_dict': opt.state_dict(),
#               'loss': loss,
#               'model_config': model_config,
#               'train_config': train_config},
#               'model_exception.pth')
#   try:
#     copyfile('model.pth', '/content/drive/My Drive/' + 'model.pth')
#   except Exception as e:
#     print(e)
# raise KeyboardInterrupt

In [0]:
from pprint import pprint
pprint(study.best_params)
pprint(study.best_value)
pprint(study.best_trial)
pprint(study.direction)
%reload_ext google.colab.data_table
study.trials_dataframe()

In [0]:
# optuna.visualization.plot_contour(study, params=['beam_width', 'beam_depth', 'bkw_beam_width'])
# optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_slice(study)
# optuna.visualization.plot_parallel_coordinate(study)  # BUG
# optuna.visualization.plot_intermediate_values(study)

# Other

In [0]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

title = 'Агент Жиго отреагировал на новость об интересе «Лиона» к футболисту «Спартака»'
text = 'Мохамед Беншенафи, агент защитника московского «Спартака» Самуэля Жиго, опроверг информацию об интересе к игроку со стороны французского футбольного клуба «Лион».'
ss = [' '.join([w.lower() for w in word_tokenize(s) if w.isalnum()]) for s in sent_tokenize(text)]
pred_title = inference(model, ds, ss[0])
print(ss)
print(title)
print(pred_title)

In [0]:
from pprint import pprint
import gensim.downloader as api
info = api.info()
pprint(info['models'].keys())
vec_model = api.load('word2vec-ruscorpora-300')

In [0]:
# https://github.com/akutuzov/webvectors/blob/master/preprocessing/rusvectores_tutorial.ipynb

# print(model.most_similar("кто"))
print(len(model.vocab))
print(list(model.vocab)[:100])
print(dir(model))

In [0]:
import spacy

In [0]:
!pip install ufal.udpipe

https://universaldependencies.org/u/pos/

In [0]:
import sys
import os
# import wget
import re
from ufal.udpipe import Model, Pipeline

'''
Этот скрипт принимает на вход необработанный русский текст 
(одно предложение на строку или один абзац на строку).
Он токенизируется, лемматизируется и размечается по частям речи с использованием UDPipe.
На выход подаётся последовательность разделенных пробелами лемм с частями речи 
("зеленый_NOUN трамвай_NOUN").
Их можно непосредственно использовать в моделях с RusVectōrēs (https://rusvectores.org).
Примеры запуска:
echo 'Мама мыла раму.' | python3 rus_preprocessing_udpipe.py
zcat large_corpus.txt.gz | python3 rus_preprocessing_udpipe.py | gzip > processed_corpus.txt.gz
'''


def num_replace(word):
    newtoken = 'x' * len(word)
    return newtoken


def clean_token(token, misc):
    """
    :param token:  токен (строка)
    :param misc:  содержимое поля "MISC" в CONLLU (строка)
    :return: очищенный токен (строка)
    """
    out_token = token.strip().replace(' ', '')
    if token == 'Файл' and 'SpaceAfter=No' in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    """
    :param lemma: лемма (строка)
    :param pos: часть речи (строка)
    :return: очищенная лемма (строка)
    """
    out_lemma = lemma.strip().replace(' ', '').replace('_', '').lower()
    if '|' in out_lemma or out_lemma.endswith('.jpg') or out_lemma.endswith('.png'):
        return None
    if pos != 'PUNCT':
        if out_lemma.startswith('«') or out_lemma.startswith('»'):
            out_lemma = ''.join(out_lemma[1:])
        if out_lemma.endswith('«') or out_lemma.endswith('»'):
            out_lemma = ''.join(out_lemma[:-1])
        if out_lemma.endswith('!') or out_lemma.endswith('?') or out_lemma.endswith(',') \
                or out_lemma.endswith('.'):
            out_lemma = ''.join(out_lemma[:-1])
    return out_lemma


def list_replace(search, replacement, text):
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text


def unify_sym(text):  # принимает строку в юникоде
    text = list_replace \
        ('\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019', '\u0022', text)

    text = list_replace \
        ('\u2012\u2013\u2014\u2015\u203E\u0305\u00AF', '\u2003\u002D\u002D\u2003', text)

    text = list_replace('\u2010\u2011', '\u002D', text)

    text = list_replace \
            (
            '\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000',
            '\u2002', text)

    text = re.sub('\u2003\u2003', '\u2003', text)
    text = re.sub('\t\t', '\t', text)

    text = list_replace \
            (
            '\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062',
            '.', text)

    text = list_replace('\u2217', '\u002A', text)

    text = list_replace('…', '...', text)

    text = list_replace('\u2241\u224B\u2E2F\u0483', '\u223D', text)

    text = list_replace('\u00C4', 'A', text)  # латинская
    text = list_replace('\u00E4', 'a', text)
    text = list_replace('\u00CB', 'E', text)
    text = list_replace('\u00EB', 'e', text)
    text = list_replace('\u1E26', 'H', text)
    text = list_replace('\u1E27', 'h', text)
    text = list_replace('\u00CF', 'I', text)
    text = list_replace('\u00EF', 'i', text)
    text = list_replace('\u00D6', 'O', text)
    text = list_replace('\u00F6', 'o', text)
    text = list_replace('\u00DC', 'U', text)
    text = list_replace('\u00FC', 'u', text)
    text = list_replace('\u0178', 'Y', text)
    text = list_replace('\u00FF', 'y', text)
    text = list_replace('\u00DF', 's', text)
    text = list_replace('\u1E9E', 'S', text)

    currencies = list \
            (
            '\u20BD\u0024\u00A3\u20A4\u20AC\u20AA\u2133\u20BE\u00A2\u058F\u0BF9\u20BC\u20A1\u20A0\u20B4\u20A7\u20B0\u20BF\u20A3\u060B\u0E3F\u20A9\u20B4\u20B2\u0192\u20AB\u00A5\u20AD\u20A1\u20BA\u20A6\u20B1\uFDFC\u17DB\u20B9\u20A8\u20B5\u09F3\u20B8\u20AE\u0192'
        )

    alphabet = list \
            (
            '\t\n\r абвгдеёзжийклмнопрстуфхцчшщьыъэюяАБВГДЕЁЗЖИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ,.[]{}()=+-−*&^%$#@!?~;:0123456789§/\|"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ')

    alphabet.append("'")

    allowed = set(currencies + alphabet)

    cleaned_text = [sym for sym in text if sym in allowed]
    cleaned_text = ''.join(cleaned_text)

    return cleaned_text


def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    # Если частеречные тэги не нужны (например, их нет в модели), выставьте pos=False
    # в этом случае на выход будут поданы только леммы
    # По умолчанию знаки пунктуации вырезаются. Чтобы сохранить их, выставьте punct=True

    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn

import requests
import os 

# URL of the UDPipe model
udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
udpipe_filename = udpipe_model_url.split('/')[-1]

if not os.path.isfile(udpipe_filename):
    print('UDPipe model not found. Downloading...', file=sys.stderr)
    r = requests.get(udpipe_model_url)
    with open(udpipe_filename, 'wb') as f:
      f.write(r.content)

print('\nLoading the model...', file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(model, 'tokenize', 
                            Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

# print('Processing input...', file=sys.stderr)
# for line in sys.stdin:
#     res = unify_sym(line.strip())
#     output = process(process_pipeline, text=res)
#     print(' '.join(output))

In [0]:
# from torch.utils.data import Dataset, IterableDataset, DataLoader, random_split
# import json

# from tqdm.notebook import tqdm

# news = []
# n_news = 1000
# with gzip.open(comp_fn, 'rb') as gz_file:
#   for _ in tqdm(range(n_news)):
#     news.append(json.loads(next(gz_file)))
# print(news[0])
# print(len(news))

# from collections import Counter

# words = Counter()
# for n in news:
#   words.update(w for s in n['text'] for w in s.split())
#   words.update(w for s in n['title'] for w in s.split())
# print(words.most_common(10))
# print(len(words))

# from torchtext.vocab import Vocab, SubwordVocab
# min_freq = 0
# max_size = 40000
# voc = Vocab(counter=words, max_size=max_size, specials=['<pad>', '<unk>', '<go>', '<eos>'])
# # voc = SubwordVocab(counter=words, max_size=max_size, specials=['<pad>', '<unk>', '<go>', '<eos>'])
# print(len(voc))
# print(dir(voc))
# print(voc.stoi)

# class RiaDataset(IterableDataset):
#   def __init__(self, path, start, end, vocab):
#     super(RiaDataset).__init__()
#     self.path = path
#     self.start = start
#     self.end = end
#     self.vocab = vocab

#   def __len__(self):
#     return self.end - self.start

#   def __iter__(self):
#     worker_info = torch.utils.data.get_worker_info()
#     # print(worker_info.id, id(self), self.start, self.end)
#     with open(self.path) as f:
#       for _ in range(self.start):
#         next(f)
#       for _ in range(self.start, self.end):
#         yield self.preprocess(next(f))

#   def preprocess(self, data):
#     data = json.loads(data)
#     enc_text = [[self.vocab.stoi.get(w, 1) for w in s.split()] for s in data['text']]
#     enc_title = [[self.vocab.stoi.get(w, 1) for w in s.split()] for s in data['title']]
#     # dec_text = [[self.vocab.itos[w] for w in s] for s in enc_text]
#     # enc_text = [[w for w in s.split()] for s in data['text']]
#     # enc_text = [w for s in data['text'] for w in s.split()]
#     # enc_title = [w for s in data['title'] for w in s.split()]
#     enc_title = [self.vocab.stoi.get(w, 1) for s in data['title'] for w in s.split()]
#     enc_text = [self.vocab.stoi.get(w, 1) for s in data['text'] for w in s.split()][:len(enc_title)*3]
#     return [enc_text, enc_title]

# def worker_init_fn(worker_id):
#   worker_info = torch.utils.data.get_worker_info()
#   s, e = worker_info.dataset.start, worker_info.dataset.end
#   n = worker_info.num_workers
#   q, r = (e - s) // n, (e - s) % n
#   s = s + worker_id * q
#   e = s + q if worker_id < n - 1 else s + q + r
#   worker_info.dataset.start, worker_info.dataset.end = s, e

# def collate_fn(batch):
#   bx1, bx2 = [], []
#   for b in batch:
#     bx1.append(torch.tensor(b[0]))
#     bx2.append(torch.tensor(b[1]))
#   bx1 = torch.nn.utils.rnn.pad_sequence(bx1, batch_first=True)
#   bx2 = torch.nn.utils.rnn.pad_sequence(bx2, batch_first=True)
#   batch = [bx1, bx2]
#   return batch

# ds = RiaDataset('norm_sents_ria_1k.json', 0, 100, voc)
# dl = DataLoader(ds, batch_size=3, num_workers=1, shuffle=False, drop_last=False, 
#                                  collate_fn=collate_fn, worker_init_fn=worker_init_fn)
# for b in tqdm(dl):
#   print(b[0].size(), b[1].size())
#   print(len(b))
# # train_len = int(0.7*len(ds))
# # test_len = int(0.2*len(ds))
# # val_len = len(ds) - train_len - test_len
# # lens = [train_len, test_len, val_len]
# # print(lens)
# # train_ds, test_ds, val_ds = random_split(ds, lens)
# # dl = DataLoader(train_ds, batch_size=2, num_workers=1, shuffle=False, drop_last=False, 
# #                                  collate_fn=collate_fn, worker_init_fn=worker_init_fn)
# # # print(dir(train_ds))
# # # print(train_ds.indices)
# # for b in tqdm(dl):
# #   print(b[0].size(), b[1].size())
# #   print(len(b))

http://wiki.nlpl.eu/index.php/Home

In [0]:
import zipfile
import os

model_url = 'http://vectors.nlpl.eu/repository/20/184.zip'  # Russian News
fn = os.path.basename(model_url)
if not os.path.exists(fn):
  r = requests.get(model_url)
  with open(fn, 'wb') as f:
    f.write(r.content)

with zipfile.ZipFile(fn) as f:
  stream = f.open('model.bin')
  vec_model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [0]:
vec_model = api.load('word2vec-ruscorpora-300')

In [0]:
text = news[0]['text'][1]
print(text)
tagged = process(process_pipeline, text)
print(tagged)

In [0]:
vec_model = api.load('word2vec-ruscorpora-300')
vs = [vec_model[x].shape for x in tagged if x in vec_model]
print(vec_model[tagged[0]])
print(len(tagged), len(vs))

In [0]:
import multiprocessing
import os

print(multiprocessing.cpu_count())
print(len(os.sched_getaffinity(0)))

http://vectors.nlpl.eu/repository/